In [13]:
import json
import joblib
from sklearn.model_selection import train_test_split
from sentence_transformers import SentenceTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

In [14]:
dataset = []
with open("aviation_cmds.jsonl", "r") as f:
    for line in f:
        dataset.append(json.loads(line))

train_data, test_data = train_test_split(dataset, test_size=0.2, random_state=42)
print(len(train_data), len(test_data))

1459 365


In [15]:
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

X_train = [d['text'] for d in train_data]
y_train = [d['intent'] for d in train_data]

X_test = [d['text'] for d in test_data]
y_test = [d['intent'] for d in test_data]

X_train_embeddings = embedding_model.encode(X_train)
X_test_embeddings = embedding_model.encode(X_test)

In [16]:
classifier = LogisticRegression(max_iter=1000)
classifier.fit(X_train_embeddings, y_train)

y_pred = classifier.predict(X_test_embeddings)
print("Test Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

Test Accuracy: 1.0
                       precision    recall  f1-score   support

      AUTOPILOT_1_OFF       1.00      1.00      1.00        15
       AUTOPILOT_1_ON       1.00      1.00      1.00        15
      AUTOPILOT_2_OFF       1.00      1.00      1.00        23
       AUTOPILOT_2_ON       1.00      1.00      1.00        18
         ENGINE_1_OFF       1.00      1.00      1.00        18
          ENGINE_1_ON       1.00      1.00      1.00        17
         ENGINE_2_OFF       1.00      1.00      1.00        18
          ENGINE_2_ON       1.00      1.00      1.00        19
           FLAPS_DOWN       1.00      1.00      1.00        23
             FLAPS_UP       1.00      1.00      1.00        22
FLIGHT_DIRECTOR_1_OFF       1.00      1.00      1.00        18
 FLIGHT_DIRECTOR_1_ON       1.00      1.00      1.00        21
FLIGHT_DIRECTOR_2_OFF       1.00      1.00      1.00        20
 FLIGHT_DIRECTOR_2_ON       1.00      1.00      1.00        21
            GEAR_DOWN       1.00   

In [17]:
def predict_action(command_text):
    embedding = embedding_model.encode([command_text])
    return classifier.predict(embedding)[0]

print(predict_action("lift the landing gear"))
print(predict_action("engage autopilot 2 now"))
print(predict_action("release parking brake"))

GEAR_UP
AUTOPILOT_2_ON
PARKING_BRAKE_OFF


In [18]:
joblib.dump(classifier, "ai_copilot.pkl")
print("Saved model as ai_copilot.pkl")

Saved model as ai_copilot.pkl
